In [ ]:
import ee
import geemap
import datetime
import time
import os
#import ollama
import notebook
from IPython.core.display_functions import display
import ipywidgets 

# run       python3 -m notebook      in shell to start Jupyter server
ee.Authenticate(auth_mode=locals,quiet=True)
ee.Initialize(project="learning-project-436517")
print(ee.String("ee, imported, auth complete").getInfo())

#========================================================================================= Polygon Start ======================================================================================================================================
Ukraine = {
    "geodesic": False,
    "type": "Polygon",
    "coordinates": [[[
        23.32989305133208, 46.18192421358463],
        [38.66680711383208, 46.18192421358463],
        [38.66680711383208, 52.081364449517274],
        [23.32989305133208, 52.081364449517274],
        [23.32989305133208, 46.18192421358463]]]}
Mariupol = {
    "geodesic": False,
    "type": "Polygon",
    "coordinates": [[
        [37.57443050213773, 47.08504167310611],
        [37.66575435467679, 47.07709281123811],
        [37.70557979412992, 47.128505634229306],
        [37.65751460858304, 47.152792595141435],
        [37.490659750184605, 47.12943995326131],
        [37.47555354901273, 47.02750256697862],
        [37.57443050213773, 47.08504167310611]]]}
Odesa = {
    "geodesic": False,
    "type": "Polygon",
    "coordinates": [
        [
            [30.598608101814754, 46.34232305677091],
            [30.801855172127254, 46.34232305677091],
            [30.801855172127254, 46.55049603225274],
            [30.598608101814754, 46.55049603225274],
            [30.598608101814754, 46.34232305677091]
        ]
    ]
}
Gaza = {
    "geodesic": False,
    "type": "coordinates",
    "coordinates": [
        [
            [34.281418764703915, 31.161562347574517],
            [34.58766266118829, 31.541541281809828],
            [34.49153229009454, 31.608230377233596],
            [34.20863434087579, 31.327112208340928],
            [34.281418764703915, 31.161562347574517]
        ]
    ]
}
Location = Odesa # redefine location for testing
#========================================================================================= Polygon End ======================================================================================================================================

#========================================================================================= Dates Start ======================================================================================================================================

py_date = datetime.date.today().isoformat()
ee_date = ee.Date(py_date.format())  # epoch format


print(f"py_date: {py_date}")
print(f"ee_date: {ee_date.getInfo()['value']}")

Today = ee_date                     #   }-Short 14 day period for "current"  Data
Past = ee_date.advance(-14, 'day')  #   }

Past1 = ee_date.advance(-30, 'day') #   }-Short 30 day period to gather "past" Data
Past2 = ee_date.advance(-60, 'day') #   }

#========================================================================================= Dates End ======================================================================================================================================

#========================================================================================= Map Start ======================================================================================================================================

Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_basemap("OpenTopoMap")
display(Map)

#========================================================================================= Map End ======================================================================================================================================


ee, imported, auth complete
py_date: 2025-07-03
ee_date: 1751500800000


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…